In [1]:
#coding utf-8

#Author: Valentas Kurauskas
#Licence: MIT

#NB: I cannot guarantee there are no more bugs here

In [2]:
#Calculate the largest component size (fraction of infected population) with given R0.
#Compares binomial model (green line) with three-point degree distribution model, in which
#an individual infects exactly 0, a or b new individuals
#with probability 1-p_a-epsilon, p_a, epsilon
#adjusted to have desired R0.

#Based on Svante Janson, and Malwina J. Luczak, "A new approach to the giant component problem", Random Structures & Algorithms 34 (2009): 197-216.

In [3]:
import matplotlib.pyplot as plt
import math
import ipywidgets
from scipy.optimize import fsolve, minimize

def xi(f, df, mu):
    if df(1) <= 1: return 0
    g = lambda x: x * mu - df(x)
    xi = fsolve(g, 0)
    if (xi < 0 or xi > 1):
        return 0
    return xi

def comp1(f, df, mu):
    if df(1) <= 1:
        return 0
    return 1 - f(xi(f,df,mu))

def cc(a,b,R0,p_a):
    if R0 >= b: #invalid parameters
        return 0
    eps = (p_a * (a-1) * (R0-a) + R0)/((b-1)*(b-R0))
    if eps < 0 or 1 - p_a - eps < 0:
        return 0 #invalid parameters
    m = (1-p_a - eps) + p_a * a + eps * b
    m2 = (1-p_a - eps) + p_a * a**2 + eps * b**2
    if (m2 - 2*m) <= 0: return 0 #epidemics does not explode
    f=lambda x: (1-p_a-eps) * x + p_a * x**a + eps * x**b
    df=lambda x: 1 - p_a - eps + a*p_a*x**(a-1) + eps * b * x**(b-1)
    res=comp1(f, df, m)[0]
    return res

def plot_toy3(a, R0, p_a, max_b=30):
    a=int(a)
    plt.bar(range(a+2, max_b+1), [cc(a+1, b+1, R0, p_a) for b in range(a+2,max_b+1)], color="gray")
    gnp = comp1(lambda x: math.exp( R0 * (x-1)), lambda x: R0 * math.exp( R0 * (x-1)), R0)
    plt.hlines(gnp, 0, max_b, color="green")
    plt.hlines(1-1/R0, 0, max_b, color="blue")
    plt.ylim(0, 1.0)
    plt.xlim(0, max_b)
    print("{:0.2%} infect {}, 100ε% (the superspreaders) infect b, others infect 0".format(p_a,int(a)))
    plt.annotate("G(n,p) with np=R0", (max_b,gnp+0.01), color="green",ha="right")
    plt.annotate("1-1/R0", (max_b,1-1/R0+0.01), color="blue", ha="right")
    plt.title("R0={}, a={}, p_a={}".format(R0,a,p_a))
    plt.xlabel("b (how many do superspreaders infect)")
    plt.ylabel("Total fraction that will get sick")
    plt.grid()
    plt.gca()

In [4]:
#y axis shows the proportion of the population that will get sick at some point, 1=100%
ipywidgets.interact(plot_toy3, a=ipywidgets.FloatSlider(min=0, max=10, step=1, value=1),
            R0=ipywidgets.FloatSlider(min=0, max=10, step=0.1, value=2.5),
            p_a=ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.3),
            max_b=ipywidgets.IntSlider(min=10, max=200, step=10, value=50)
         );

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, step=1.0), FloatSlider(value=2.5, desc…

For $G(n,p)$ the blue line is the ["herd immunity threshold"](https://en.wikipedia.org/wiki/Herd_immunity#Mechanics) $1 - \frac 1 {R_0}$. If herd immunity is attained via "natural infection" as opposed to vaccination, the epidemics only begins to die out once this point is reached. The total infected fraction (the green line) is quite a bit larger, it is the unique solution $\rho=\rho(R_0)$ of $1-\rho = e^{-R_0 \rho}$
in the interval $0 < \rho < 1$.

In [5]:
def H(a,b,R0,p_a):
    empty=(1, lambda x: 0, lambda x: 0, lambda x:0)
    if R0 >= b: 
        print("Error: R0 should be smaller than b")
        return empty
    eps = (p_a * (a-1) * (R0-a) + R0)/((b-1)*(b-R0))
    if eps < 0 or 1 - p_a - eps < 0:
        print("Error: These parameters are invalid")
        return empty
    m = (1-p_a - eps) + p_a * a + eps * b
    m2 = (1-p_a - eps) + p_a * a**2 + eps * b**2
    print("{:0.2%} infect 0, {:0.2%} infect {}, {:0.2%} infect {}".format(1-p_a-eps, p_a,int(a-1), eps,b-1))
    if (m2 - 2*m) <= 0:
        print("Epidemics ends immediately!")
        return empty
    f = lambda x: (1-p_a-eps) * x + p_a * x**a + eps * x**b
    df = lambda x: 1 - p_a - eps + a*p_a*x**(a-1) + eps * b * x**(b-1)
    d2 = lambda x: a*(a-1) * p_a * x**a + eps * b * (b-1) * x**b #  from (5.1)
    Ed = df(1)
    return xi(f,df,m), lambda x: Ed * x ** 2 - x * df(x), lambda x: d2(x)/(x*df(x)), f

In [6]:
def plot_H(a, R0, p_a, b):
    xi, f, Rt, g = H(a+1, b+1, R0, p_a)
    xs = [ -math.log(xi) * i/100 for i in range(101)]
    plt.figure(figsize=(5, 6))
    plt.subplot(311)
    plt.plot(xs, [f(math.exp(-x)) for x in xs], "b-", label="active")
    plt.legend()
    plt.grid()
    plt.subplot(312)
    plt.plot(xs, [1-g(math.exp(-x)) for x in xs], "g-", label="infected/removed")
    plt.legend()
    plt.grid()
    plt.subplot(313)
    plt.plot(xs, [Rt(math.exp(-x)) for x in xs], "r-", label="Rt")
    plt.hlines(1, 0, -math.log(xi), linestyles="dotted")
    plt.xlabel("time (distorted)")
    plt.grid()
    plt.legend()
    plt.gca()

In [7]:
#"Superspreaders" cause large Rt in the beginning but are explored sooner.
#Note that the time is (monotonically) transformed, it does not match the real time
#y axis shows the proportion of the population, 1=100%
ipywidgets.interact(plot_H, a=ipywidgets.FloatSlider(min=0, max=10, step=1, value=1),
            R0=ipywidgets.FloatSlider(min=0, max=10, step=0.1, value=2.5),
            p_a=ipywidgets.FloatSlider(min=0, max=1, step=0.01, value=0.3),
            b=ipywidgets.IntSlider(min=2, max=100, step=1, value=10));

interactive(children=(FloatSlider(value=1.0, description='a', max=10.0, step=1.0), FloatSlider(value=2.5, desc…